# Load Libraries/Data

In [66]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"
akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"

In [38]:
basics_df = pd.read_csv(basics_url, sep='\t', low_memory=False)

In [39]:
basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [40]:
ratings_df = pd.read_csv(ratings_url, sep='\t', low_memory=False)

In [41]:
ratings_df.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1982
1,tt0000002,5.8,265
2,tt0000003,6.5,1838
3,tt0000004,5.5,178
4,tt0000005,6.2,2625


In [42]:
akas_df = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [43]:
akas_df.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


# Clean Data

## Basics Data

### Replace '\N' with np.nan

In [44]:
basics_df = basics_df.replace({'\\N':np.nan})

In [45]:
basics_df[basics_df.eq('\\N').any(1)]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


In [46]:
basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


### Eliminate movies that are null for runtimeMinutes

In [47]:
basics_df.dropna(subset=['runtimeMinutes'], inplace=True)

In [48]:
basics_df['runtimeMinutes'].isna().sum()

0

### Eliminate movies that are null for genre

In [49]:
basics_df.dropna(subset=['genres'], inplace=True)

In [50]:
basics_df['genres'].isna().sum()

0

### Keep only titleType==Movie

In [51]:
basics_df['titleType'].unique()

array(['short', 'movie', 'tvShort', 'tvMovie', 'tvSeries', 'tvEpisode',
       'tvMiniSeries', 'video', 'tvSpecial', 'videoGame'], dtype=object)

In [52]:
is_movie = basics_df['titleType'] == 'movie'
basics_df = basics_df[is_movie]
basics_df['titleType'].unique()

array(['movie'], dtype=object)

In [53]:
basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,NaN,100,"Documentary,News,Sport"
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"


### Keep startYear 2000-2022

In [54]:
basics_df['startYear'].info()

<class 'pandas.core.series.Series'>
Int64Index: 384294 entries, 8 to 9958306
Series name: startYear
Non-Null Count   Dtype 
--------------   ----- 
377783 non-null  object
dtypes: object(1)
memory usage: 5.9+ MB


In [56]:
year_range = (basics_df['startYear'] >= '2000') & (basics_df['startYear'] <= '2022')
basics_df = basics_df[year_range]
basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
13082,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021,NaN,94,Documentary
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
42384,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013,NaN,120,"Drama,History"
61115,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67667,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama


### Eliminate movies that include "Documentary" in genre

In [59]:
# Exclude movies that are included in the documentary category.
is_documentary = basics_df['genres'].str.contains('documentary',case=False)
basics_df = basics_df[~is_documentary]

In [60]:
basics_df[is_documentary]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


### Keep only US movies

In [78]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics_df['tconst'].isin(akas_df['titleId'])
keepers

34803       True
42384       True
61115       True
67667       True
86799       True
           ...  
9958029     True
9958038     True
9958077    False
9958122     True
9958206    False
Name: tconst, Length: 147139, dtype: bool

In [79]:
basics_df = basics_df[keepers]
basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
42384,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013,NaN,120,"Drama,History"
61115,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67667,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
86799,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


## AKAs Data

### Keep Only US Movies

In [63]:
akas_df.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [75]:
us_only = akas_df['region'] == 'US'

In [77]:
akas_df = akas_df[us_only]
akas_df

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,\N,imdbDisplay,\N,0
14,tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,\N,imdbDisplay,\N,0
36,tt0000005,1,Blacksmithing Scene,US,\N,alternative,\N,0
41,tt0000005,6,Blacksmith Scene #1,US,\N,alternative,\N,0
...,...,...,...,...,...,...,...,...
36351175,tt9916560,1,March of Dimes Presents: Once Upon a Dime,US,\N,imdbDisplay,\N,0
36351245,tt9916620,1,The Copeland Case,US,\N,imdbDisplay,\N,0
36351334,tt9916702,1,Loving London: The Playground,US,\N,\N,\N,0
36351377,tt9916756,1,Pretty Pretty Black Girl,US,\N,imdbDisplay,\N,0


### Replace "\N" with np.nan

In [80]:
akas_df.replace({'\\N':np.nan}, inplace=True)
akas_df[akas_df.eq('\\N').any(1)]

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle


## Ratings Data

### Replace "\N" with np.nan (if any)

In [81]:
ratings_df[ratings_df.eq('\\N').any(1)]

,tconst,averageRating,numVotes


### Keep only US movies

In [82]:
ratings_df.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1982
1,tt0000002,5.8,265
2,tt0000003,6.5,1838
3,tt0000004,5.5,178
4,tt0000005,6.2,2625


In [83]:
ratings_us = ratings_df['tconst'].isin(akas_df['titleId'])

In [84]:
ratings_df = ratings_df[ratings_us]
ratings_df

,tconst,averageRating,numVotes
0,tt0000001,5.7,1982
1,tt0000002,5.8,265
4,tt0000005,6.2,2625
5,tt0000006,5.1,182
6,tt0000007,5.4,820
...,...,...,...
1323507,tt9916200,8.1,230
1323508,tt9916204,8.2,263
1323515,tt9916348,8.3,18
1323516,tt9916362,6.4,5388


# Data .info()

In [85]:
basics_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86819 entries, 34803 to 9958122
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          86819 non-null  object
 1   titleType       86819 non-null  object
 2   primaryTitle    86819 non-null  object
 3   originalTitle   86819 non-null  object
 4   isAdult         86819 non-null  object
 5   startYear       86819 non-null  object
 6   endYear         0 non-null      object
 7   runtimeMinutes  86819 non-null  object
 8   genres          86819 non-null  object
dtypes: object(9)
memory usage: 6.6+ MB


In [86]:
akas_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1447035 entries, 5 to 36351393
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1447035 non-null  object
 1   ordering         1447035 non-null  int64 
 2   title            1447035 non-null  object
 3   region           1447035 non-null  object
 4   language         3960 non-null     object
 5   types            980456 non-null   object
 6   attributes       46840 non-null    object
 7   isOriginalTitle  1445693 non-null  object
dtypes: int64(1), object(7)
memory usage: 99.4+ MB


In [87]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501968 entries, 0 to 1323521
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         501968 non-null  object 
 1   averageRating  501968 non-null  float64
 2   numVotes       501968 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.3+ MB


# Create 'Data' Folder

In [88]:
# example making new folder with os
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

[]

## Save to Folder

In [89]:
basics_df.to_csv('Data/title_basics.csv.gz', compression='gzip', index=False)

In [90]:
akas_df.to_csv('Data/title_akas.csv.gz', compression='gzip', index=False)

In [91]:
ratings_df.to_csv('Data/title_ratings.csv.gz', compression='gzip', index=False)

## Replace DataFrame Object

In [92]:
basics_df = pd.read_csv('Data/title_basics.csv.gz', low_memory=False)
basics_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013,NaN,120,"Drama,History"
2,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
86814,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
86815,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy"
86816,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
86817,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"


In [93]:
akas_df = pd.read_csv('Data/title_akas.csv.gz', low_memory=False)
akas_df

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0
...,...,...,...,...,...,...,...,...
1447030,tt9916560,1,March of Dimes Presents: Once Upon a Dime,US,NaN,imdbDisplay,NaN,0.0
1447031,tt9916620,1,The Copeland Case,US,NaN,imdbDisplay,NaN,0.0
1447032,tt9916702,1,Loving London: The Playground,US,NaN,NaN,NaN,0.0
1447033,tt9916756,1,Pretty Pretty Black Girl,US,NaN,imdbDisplay,NaN,0.0


In [94]:
ratings_df = pd.read_csv('Data/title_ratings.csv.gz', low_memory=False)
ratings_df

,tconst,averageRating,numVotes
0,tt0000001,5.7,1982
1,tt0000002,5.8,265
2,tt0000005,6.2,2625
3,tt0000006,5.1,182
4,tt0000007,5.4,820
...,...,...,...
501963,tt9916200,8.1,230
501964,tt9916204,8.2,263
501965,tt9916348,8.3,18
501966,tt9916362,6.4,5388
